<a href="https://colab.research.google.com/github/mihirsathe/ECE285FA18_BestGroup/blob/master/Keras_OvrSR_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip3 install -q git+https://www.github.com/keras-team/keras-contrib.git
#!pip3 install -q DataLoader
#!pip3 install -q imageio
#!pip3 install -q keras

#Mount google drive with dataset
#from google.colab import drive
#drive.mount('/content/drive')
dir_pfx = './'
data_dir = '../../data/Vehicules1024/'
# keras.backend.set_session

In [2]:
# Super-resolution of CelebA using Generative Adversarial Networks.
# The dataset can be downloaded from: https://www.dropbox.com/sh/8oqt9vytwxb3s4r/AADIKlz8PR9zr6Y20qbkunrba/Img/img_align_celeba.zip?dl=0
# Instrustion on running the script:
# 1. Download the dataset from the provided link
# 2. Save the folder 'img_align_celeba' to 'datasets/'
# 4. Run the sript using command 'python srgan.py'


from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils.training_utils import multi_gpu_model

import datetime
import matplotlib.pyplot as plt
import sys
# from google.col
import numpy as np
import os

import keras.backend as K
import glob

Using TensorFlow backend.


In [3]:
import numpy as np
import imageio
from keras.utils import Sequence
from os import listdir
from os.path import isfile, join
from skimage.transform import downscale_local_mean
import random
import tensorflow as tf
import scipy


def normalize(image):
  return image/255.0    

def read_VEDAI(subset, PATH_TO_VEHICLES_FOLDER):
    # Takes in the full path to the unzipped "VEHICULES" folder
    # Returns mapping dict, RGB and Infrared images in 
    # (images, x, y, channels) format,
    # saves a txt file with mapping of rgb/infra idx to filename
    # NUM_FILES = 2536
    # MAX_INDEX = 1272
    X_PIXELS = 1024
    Y_PIXELS = 1024
    PATH_TO_VEHICLES_FOLDER = PATH_TO_VEHICLES_FOLDER#[0]

    onlyfiles = [f for f in listdir(PATH_TO_VEHICLES_FOLDER) if isfile(
        join(PATH_TO_VEHICLES_FOLDER, f)) and "png" in f]
    # assert len(onlyfiles) == NUM_FILES, "Not the full VEDAI 1024 Dataset"
    rgb = np.zeros((len(subset), X_PIXELS, Y_PIXELS, 3))
    infra = np.zeros((len(subset), X_PIXELS, Y_PIXELS, 1))
    indices = subset

    print(indices)
    print(rgb.shape)
    index_filename_map = {}
    im_cnt = 0

    for index in indices:
        pair = [file for file in onlyfiles if str(index) in file]
        if pair:
            for file in pair:
                index_filename_map[im_cnt] = file.split('_')[0]
                im = imageio.imread(PATH_TO_VEHICLES_FOLDER + '/' + file)
                if "co" in file:
                    # print('Inserting RGB @ '+ str(im_cnt))
                    rgb[im_cnt, :, :, :] = np.reshape(
                        im, (tuple([1]) + im.shape))
                elif "ir" in file:
                    # print('Inserting Infra @ '+ str(im_cnt))
                    infra[im_cnt, :, :, :] = np.reshape(
                        im, (tuple([1]) + im.shape + tuple([1])))
            im_cnt = im_cnt + 1
        else:
            print("The following image is missing!: " + index)
    # print(index_filename_map)
    f = open(PATH_TO_VEHICLES_FOLDER + "_mapping.txt", "w")
    f.write(str(index_filename_map))
    f.close()
    return rgb, infra


def scan_dataset(PATH_TO_VEHICLES_FOLDER, number_of_imgs):
    # Takes in the full path to the unzipped "VEHICULES" folder
    # Returns a list of all the files
    # and saves a dataset summary text file with list of all file names
    MAX_INDEX = number_of_imgs
    PATH_TO_VEHICLES_FOLDER = PATH_TO_VEHICLES_FOLDER#[0]
    indices = [format(n, '08') for n in range(MAX_INDEX)]
    export_files = []

    onlyfiles = [f for f in listdir(PATH_TO_VEHICLES_FOLDER) if isfile(
        join(PATH_TO_VEHICLES_FOLDER, f)) and "png" in f]

    for index in indices:
        pair = [file for file in onlyfiles if str(index) in file]
        if pair:
            for file in pair:
                if "co" in file:
                    export_files.append(file.split('_')[0])
        else:
            print("The following image is missing!: " + index)
    np.savetxt(PATH_TO_VEHICLES_FOLDER + '_summary.txt',
               export_files, delimiter=" ", fmt="%s")
    return export_files


def create_subsets(imgs, output_path, use_validation=True,
                   training_percent=0.7, testing_percent=0.3, SEED=1):
    # Takes a list of image file names and shuffles
    # them before splitting them into required subsets
    # Saves txt files containing the names of the files
    # used in each subset, no return value

    assert training_percent + \
        testing_percent == 1, "Training + testing percents must equal 1."
    random.seed(SEED)
    random.shuffle(imgs)
    print('Using ' + str(len(imgs)) + ' images.')
    print('Saving files to ' + output_path)
    if not use_validation:
        training_imgs = imgs[:int(len(imgs) * training_percent)]
        testing_imgs = imgs[int(len(imgs) * training_percent):]
        np.savetxt(output_path + 'training.txt',
                   training_imgs, delimiter=" ", fmt="%s")
        np.savetxt(output_path + 'testing.txt',
                   testing_imgs, delimiter=" ", fmt="%s")
        return training_imgs, testing_imgs
    else:
        validation_split = 0.3  # use 30% of training dataset for validation
        training_imgs = imgs[int(len(imgs) * validation_split *
                                 training_percent):int(len(imgs) *
                                                       training_percent)]
        validation_imgs = imgs[:int(
            len(imgs) * validation_split * training_percent)]
        testing_imgs = imgs[int(len(imgs) * training_percent):]
        np.savetxt(output_path + 'validation.txt',
                   validation_imgs, delimiter=" ", fmt="%s")
        np.savetxt(output_path + 'training.txt',
                   testing_imgs, delimiter=" ", fmt="%s")
        np.savetxt(output_path + 'testing.txt',
                   testing_imgs, delimiter=" ", fmt="%s")
        return training_imgs, validation_imgs, testing_imgs


def save_VEDAI(rgb, infra):
    # Takes in arrays of rgb and infrared images
    # Saves them to disk, no return value
    np.save("vedai_rgb_all.npy", rgb)
    np.save("vedai_infra_all.npy", infra)


def load_VEDAI():
    # No parameters, expected to run in directory with VEDAI.npy files
    # Returns two arrays with rgb and infrared images respectively
    rgb = np.load("vedai_rgb_all.npy")
    infra = np.load("vedai_infra_all.npy")
    return rgb, infra


def data_explore(data):
    print("Shape of the data is" + str(data.shape))
    print("Dtype of the data is" + str(data.dtype))


def combine_rgb_infra(rgb, infra):
    # Concatenates the two modalities along the channels axis
    four_channel = np.concatenate((rgb, infra), axis=-1)
    return four_channel

  
def overlapping_patches(images, patch_size=(64, 64), padding="VALID"):
    sess = tf.Session()

    num_images, size_x, size_y, channels = images.shape
    ims = tf.convert_to_tensor(images)
    patch_x, patch_y = patch_size
    patches = tf.extract_image_patches(ims, [1, patch_x, patch_y, 1], [
        1, patch_x, patch_y, 1], [1, 1, 1, 1], padding=padding)
    patches_shape = tf.shape(patches)
    with sess.as_default():
        np = tf.reshape(patches, [tf.reduce_prod(patches_shape[0:3]),
                                  patch_x, patch_y, channels]).eval()
        return np  


def non_overlapping_patches(image, patch_size=(64, 64)):
    size_x, size_y, channels = image.shape
    patch_x, patch_y = patch_size
    im_pad = np.pad(image, ((0, patch_x - size_x % patch_x),
                            (0, patch_y - size_y % patch_y), (0, 0)),
                    mode="constant")
    if size_x % patch_x == 0 and size_y % patch_y == 0:
        im_pad = image
    pad_x, pad_y, channels = im_pad.shape
    print(im_pad.shape)
    num_patches = (pad_x // patch_x) * (pad_y // patch_y)
    patches = np.zeros((num_patches, patch_x, patch_y, channels))
    counter = 0
    for i in range((pad_x // patch_x)):
        for j in range((pad_y // patch_y)):
            x_s = i * patch_x
            y_s = j * patch_y
            patches[counter, :, :, :] = im_pad[x_s:x_s + patch_x,
                                               y_s:y_s + patch_y, :]
            counter += 1
    return patches


def downsample_image(image, block=(4, 4, 1)):
    # Downsamples numpy array image by factor
    # Returns  the downsampled copy
    if image.ndim == 4:
      block=(1, 4, 4, 1)
    return downscale_local_mean(image, block)


def reconstruct_patches(patches, image_size):
    # TODO: Create a function which reconstructs an image
    # when given patches created by non_overlapping_patches
    # Discards predictions for zero border
    pass

def get_images_to_four_chan(img_name, DATASET_PATH, ch_num=4):
  #co = imageio.imread(DATASET_PATH + 'VEDAI/' + img_name + '_co.png')
  co = imageio.imread(DATASET_PATH + img_name + '_co.png')
  if ch_num == 4:
    #ir = imageio.imread(DATASET_PATH + 'VEDAI/' + img_name + '_ir.png')
    ir = imageio.imread(DATASET_PATH + img_name + '_ir.png')
    rgb = np.reshape(co, (tuple([1]) + co.shape))
    infra = np.reshape(ir, (tuple([1]) + ir.shape + tuple([1])))  
    return combine_rgb_infra(rgb, infra)
  elif ch_num == 3:
    return np.reshape(co, (tuple([1]) + co.shape))

def load_data(file_idx, txt_file, DATASET_PATH, batch_size=1):
  # read in batch of file names from txt file with randomized filenames
  # return the lr and hr patches
  
  #read x lines from txt file
  text_file = open(DATASET_PATH +"/training.txt", "r")
  img_files = text_file.read().split('\n')
  text_file.close()

# TODO: preallocate arrays for speed  
  # Number of patches * ims_per_batch
  batchsz = 256 * batch_size
  # RGB
  channels = 3
  # Default patch size
  patch_x, patch_y = 64, 64

  # Preallocate arrays of the correct size
  imgs_hr = np.zeros((batchsz, patch_x, patch_y, channels))
  
  # Batch number * ims_per_batch
  start = file_idx
  end = file_idx + batch_size
      
#   print(start, end)    
  
  im_num = 0
  for i in range(start, end):
      st, stp = im_num * 256, (im_num + 1) * 256
      im_num += 1 
      patch = overlapping_patches(
          normalize(get_images_to_four_chan(img_files[i], DATASET_PATH, channels)))
      imgs_hr[st:stp, :, :, :] = patch
      
  imgs_lr = np.asarray([downsample_image(patch) for patch in imgs_hr])
      
  file_idx = file_idx + batch_size # update current file_idx
  return imgs_hr, imgs_lr, file_idx


class VEDAISequence(Sequence):

    def __init__(self, rgb, infra, ims_per_batch):
        self.r, self.i = rgb, infra
        self.ims_per_batch = ims_per_batch

    def __len__(self):
        # Returns number of batches given training set and ims_per_batch
        return int(np.ceil(len(self.r) / float(self.ims_per_batch)))

    def __getitem__(self, idx):
        # Number of patches * ims_per_batch
        batchsz = 256 * self.ims_per_batch
        # RGB and infra
        channels = 4
        # Default patch size
        patch_x, patch_y = 64, 64

        # Batch number * ims_per_batch
        start = idx * self.ims_per_batch
        # Batch number * ims_per_batch  + 1
        end = (idx + 1) * self.ims_per_batch

        # Preallocate arrays of the correct size
        high_res = np.zeros((batchsz, patch_x, patch_y, channels))
        im_num = 0
        for ind in range(start, end):
            st, stp = im_num * 256, (im_num + 1) * 256
            im_num += 1
            rgb = imageio.imread(self.r[ind])
            infra = imageio.imread(self.i[ind])
            high_res[st:stp, :, :, :] = non_overlapping_patches(
                combine_rgb_infra(rgb, infra))
        low_res = np.asarray([downsample_image(patch) for patch in high_res])
        return low_res, high_res

In [4]:
#path = glob.glob('./drive/My Drive/ECE285_Proj/datasets/VEDAI/*')
#files = scan_dataset(path, 25)
files = scan_dataset(data_dir, 50) #TODO: 50
#training_set, testing_set = create_subsets(files,'./drive/My Drive/ECE285_Proj/datasets/VEDAI/', use_validation = False)
training_set, testing_set = create_subsets(files,data_dir, use_validation = False)

The following image is missing!: 00000005
The following image is missing!: 00000023
Using 48 images.
Saving files to ../data/Vehicules1024/


In [5]:

class SRGAN():
    def __init__(self,num_gpus=None, weights_path=None):
        # Input shape
        self.channels = 3                  # RGB & Infra
        self.lr_height = 16                 # Low resolution height
        self.lr_width = 16                 # Low resolution width
        self.lr_shape = (self.lr_height, self.lr_width, self.channels)
        self.hr_height = self.lr_height*4   # High resolution height
        self.hr_width = self.lr_width*4     # High resolution width
        self.hr_shape = (self.hr_height, self.hr_width, self.channels)
        self.num_gpus = num_gpus
        self.weights_path = weights_path


        # Number of residual blocks in the generator
        self.n_residual_blocks = 16

        optimizer = Adam(0.0002, 0.5)

        # We use a pre-trained VGG19 model to extract image features from the high resolution
        # and the generated high resolution images and minimize the mse between them
        self.vgg = self.build_vgg()
        self.vgg.trainable = False
        self.vgg.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        
        # Calculate output shape of D (PatchGAN)
        patch = int(self.hr_height / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 16
        self.df = 16

        # Build and compile the discriminator (Keep off GPU)
        self.disc_model = self.build_discriminator()
        
        if self.num_gpus is not None:
            self.discriminator = multi_gpu_model(self.disc_model, gpus=self.num_gpus)
        
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        
        #Configure data loader name, is this still needed?
        self.dataset_name = 'VEDAI'

        # Build the generator
        self.generator = self.build_generator()

        # High res. and low res. images
        img_hr = Input(shape=self.hr_shape)
        img_lr = Input(shape=self.lr_shape)

        # Generate high res. version from low res.
        fake_hr = self.generator(img_lr)

        # Extract image features of the generated img
        fake_features = self.vgg(fake_hr)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminator determines validity of generated high res. images
        validity = self.discriminator(fake_hr)

        self.combined = Model([img_lr, img_hr], [validity, fake_features])
        
        if self.num_gpus is not None:
            self.combined = multi_gpu_model(self.combined, gpus=self.num_gpus)       
        
        self.combined.compile(loss=['binary_crossentropy', 'mse'],
                              loss_weights=[1e-3, 1],
                              optimizer=optimizer)

    def build_vgg(self):
        """
        Builds a pre-trained VGG19 model that outputs image features extracted at the
        third block of the model
        """
        vgg = VGG19(weights='imagenet')
        # Set outputs to outputs of last conv. layer in block 3
        # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
        vgg.outputs = [vgg.layers[9].output]

        img = Input(shape=self.hr_shape)
       
        # Extract image features
        img_features = vgg(img)

        return Model(img, img_features)

    def build_generator(self):
    
        def residual_block(layer_input, filters):
            """Residual block described in paper"""
            d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(layer_input)
            d = Activation('relu')(d)
            d = BatchNormalization(momentum=0.8)(d)
            d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
            d = BatchNormalization(momentum=0.8)(d)
            d = Add()([d, layer_input])
            return d

        def deconv2d(layer_input):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(64, kernel_size=3, strides=1, padding='same')(u)
            u = Activation('relu')(u)
            return u

        # Low resolution image input
        img_lr = Input(shape=self.lr_shape)

        # Pre-residual block
        c1 = Conv2D(16, kernel_size=9, strides=1, padding='same')(img_lr)
        c1 = Activation('relu')(c1)

        # Propogate through residual blocks
        r = residual_block(c1, self.gf)
        for _ in range(self.n_residual_blocks - 1):
            r = residual_block(r, self.gf)

        # Post-residual block
        c2 = Conv2D(16, kernel_size=3, strides=1, padding='same')(r)
        c2 = BatchNormalization(momentum=0.8)(c2)
        c2 = Add()([c2, c1])

        # Upsampling
        u1 = deconv2d(c2)
        u2 = deconv2d(u1)

        # Generate high resolution output
        gen_hr = Conv2D(self.channels, kernel_size=9, strides=1, padding='same', activation='tanh')(u2)
        
        generator = Model(img_lr, gen_hr)
        
        if self.weights_path is not None:
            print('Loading generator weights...')
            generator.load_weights(self.weights_path + 'OvrSR_generator_weights.h5')
            print('Loaded...')
            
        return Model(img_lr, gen_hr)

    def build_discriminator(self):

        def d_block(layer_input, filters, strides=1, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        # Input img
        d0 = Input(shape=self.hr_shape)

        d1 = d_block(d0, self.df, bn=False)
        d2 = d_block(d1, self.df, strides=2)
        d3 = d_block(d2, self.df*2)
        d4 = d_block(d3, self.df*2, strides=2)
        d5 = d_block(d4, self.df*4)
        d6 = d_block(d5, self.df*4, strides=2)
        d7 = d_block(d6, self.df*8)
        d8 = d_block(d7, self.df*8, strides=2)

        d9 = Dense(self.df*4)(d8)
        d10 = LeakyReLU(alpha=0.2)(d9)
        validity = Dense(1, activation='sigmoid')(d10)

        return Model(d0, validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()
        losses = []

        for epoch in range(epochs):

            # ----------------------
            #  Train Discriminator
            # ----------------------

            # Sample images and their conditioning counterparts
            batch_idx = 0
            imgs_hr, imgs_lr, batch_idx = load_data(batch_idx, training_set, data_dir, batch_size)

            # From low res. image generate high res. version
            fake_hr = self.generator.predict(imgs_lr)

            valid = np.ones((imgs_hr.shape[0],) + self.disc_patch)
            fake = np.zeros((imgs_hr.shape[0],) + self.disc_patch)

            # Train the discriminators (original images = real / generated = Fake)
            d_loss_real = self.discriminator.train_on_batch(imgs_hr, valid)
            d_loss_fake = self.discriminator.train_on_batch(fake_hr, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ------------------
            #  Train Generator
            # ------------------

            # Sample images and their conditioning counterparts
            imgs_hr, imgs_lr, batch_idx = load_data(batch_idx, training_set,
                      data_dir,
                      batch_size)
    
            # The generators want the discriminators to label the generated images as real
            valid = np.ones((imgs_hr.shape[0],) + self.disc_patch)

            # Extract ground truth image features using pre-trained VGG19 model
            image_features = self.vgg.predict(imgs_hr)

            # Train the generators
            g_loss = self.combined.train_on_batch([imgs_lr, imgs_hr], [valid, image_features])

            elapsed_time = datetime.datetime.now() - start_time
            # Plot the progress
            print ("%d time: %s" % (epoch, elapsed_time))
            print("\tG-Loss: " + str(g_loss) + "\tD-Loss: " + str(d_loss))
            
            # Append current losses and save
            
            losses.append([epoch] + list(g_loss) + list(d_loss))
            
            np.save(dir_pfx + 'loss_log.npy', arr=np.array(losses))            
            
            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save(self.weights_path + 'OvrSR_generator_weights.h5')
                self.disc_model.save(self.weights_path + 'OvrSR_discriminator_weights.h5')

    def sample_images(self, epoch):
        os.makedirs(dir_pfx + 'images/{0}'.format(self.dataset_name), exist_ok=True)
        r, c = 2, 2
        
        batch_idx = 0
        #imgs_hr, imgs_lr, batch_idx = load_data(batch_idx, testing_set, './drive/My Drive/ECE285_Proj/datasets/VEDAI/')
        imgs_hr, imgs_lr, batch_idx = load_data(batch_idx, testing_set, data_dir)
        fake_hr = self.generator.predict(imgs_lr)

        # Rescale images 0 - 1
        imgs_lr = 0.5 * imgs_lr + 0.5
        fake_hr = 0.5 * fake_hr + 0.5
        imgs_hr = 0.5 * imgs_hr + 0.5

        # Save generated images and the high resolution originals
        titles = ['Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        fig.suptitle('Epoch %d' % (epoch), fontsize=16)
        for row in range(r):
            for col, image in enumerate([fake_hr, imgs_hr]):
                axs[row, col].imshow(image[row])
                axs[row, col].set_title(titles[col])
                axs[row, col].axis('off')
            cnt += 1
        #fig.savefig("./drive/My Drive/ECE285_Proj/images/%s%d.png" % (self.dataset_name, epoch))
        fig.savefig(dir_pfx + 'images/{0}{1}.png'.format(self.dataset_name, epoch))
        plt.close()

        # Save low resolution images for comparison
        for i in range(r):
            fig = plt.figure()
            plt.imshow(imgs_lr[i])
            #fig.savefig('./drive/My Drive/ECE285_Proj/images/%s/%d_lowres%d.png' % (self.dataset_name, epoch, i))
            fig.savefig(dir_pfx + 'images/{0}/{1}_lowres{2}.png'.format(self.dataset_name, epoch, i))
            plt.close()

#if __name__ == '__main__':
#    print("Init model...")
#    gan = SRGAN(num_gpus=8, weights_path='./')
#    print("Starting training...")
#    gan.train(epochs=1000, batch_size=4, sample_interval=5)
#     gan.train(epochs=300, batch_size=1, sample_interval=50)

In [ ]:
print("Init model...")
gan = SRGAN(num_gpus=8, weights_path='./')
print("Starting training...")
gan.train(epochs=1000, batch_size=4, sample_interval=5)

Init model...
Loading generator weights...
Loaded...
Starting training...


/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 time: 0:04:44.808922
	G-Loss: [13.9393635, 0.84455097, 13.938519]	D-Loss: [0.2979524  0.47912598]
1 time: 0:10:27.511441
	G-Loss: [13.939474, 0.9394254, 13.938535]	D-Loss: [0.25528893 0.5744629 ]
2 time: 0:12:46.689204
	G-Loss: [14.015808, 1.0837474, 14.014725]	D-Loss: [0.2147986 0.684906 ]
3 time: 0:15:11.493832
	G-Loss: [13.349136, 1.289816, 13.347847]	D-Loss: [0.17503083 0.7739563 ]
4 time: 0:17:42.867887
	G-Loss: [13.252498, 1.4348549, 13.251062]	D-Loss: [0.13868488 0.85324097]
5 time: 0:20:12.317777
	G-Loss: [12.973996, 1.6276083, 12.972368]	D-Loss: [0.10956345 0.90252686]
6 time: 0:23:23.961883
	G-Loss: [12.754023, 1.756727, 12.752266]	D-Loss: [0.08253214 0.94763184]
7 time: 0:26:03.019418
	G-Loss: [12.620253, 1.874224, 12.618379]	D-Loss: [0.06503311 0.96676636]
8 time: 0:28:46.471903
	G-Loss: [12.485141, 2.0052433, 12.483135]	D-Loss: [0.04993654 0.98257446]
9 time: 0:31:32.836518
	G-Loss: [12.371553, 2.0914702, 12.369462]	D-Loss: [0.04060389 0.9878845 ]
10 time: 0:34:25.675448